# API para usar las funciones de predicción de Falcon

In [ ]:
# ================================
# 1. Instalar dependencias
# ================================
!pip install transformers accelerate
!npm install -g localtunnel

In [2]:
# ================================
# 2. Importar librerías necesarias
# ================================
import threading
import time
from flask import Flask, request, jsonify
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch, re

In [ ]:
# ================================
# 3. Cargar el modelo ya fusionado (sin LoRA)
# ================================
model_path = "/content/drive/MyDrive/Colab Notebooks/ft_falcon_model/final_model/falcon_merged"

# Montar directorio de drive
from google.colab import drive
drive.mount('/content/drive')

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    device_map="auto",               # permite usar parcialmente la GPU + CPU
    torch_dtype=torch.float16,      # reduce el uso de memoria
    low_cpu_mem_usage=True
)

pipe = pipeline(
        task="text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=3,
        do_sample=False,
        return_full_text=False,
        truncation=True,
        pad_token_id=tokenizer.eos_token_id,
    )

# ================================
# 4. Crear servidor Flask
# ================================
app = Flask(__name__)

@app.route("/predict", methods=["POST"])
def predict():
  data = request.get_json()
  prompt = data.get("prompt", "")
  result = pipe(prompt)
  if not result or 'generated_text' not in result[0]:
    label = 'view'
    return jsonify({"intention": label})


  text = re.sub(r'[^a-z]', ' ', result[0]['generated_text'].strip().lower()).strip()
  words = text.split()

  if not words:  # Verificar si la lista está vacía
    label = 'view'
    return jsonify({"intention": label})
  text = words[0]  # Tomar solo la primera palabra

  # Clasificación
  if 'clear' in text:
    label = 'clear'
  elif 'view' in text:
    label = 'view'
  elif 'add' in text:
    label = 'add'
  elif 'remove' in text:
    label = 'remove'
  else:
    label = 'view'
  return jsonify({"intention": label})

# ================================
# 5. Ejecutar Flask y LocalTunnel
# ================================
def run_flask():
    app.run(port=5001)

def run_tunnel():
    get_ipython().system_raw('lt --port 5001 > tunnel.txt 2>&1 &')
    time.sleep(4)
    !curl -s http://localhost:4040/api/tunnels | grep -o '"public_url":"[^"]*' | grep -o 'http[^"]*'

threading.Thread(target=run_flask).start()
time.sleep(2)
run_tunnel()
time.sleep(4)

In [ ]:
# Relanzar el túnel y mostrar la salida completa
import time
get_ipython().system_raw('lt --port 5001 > tunnel.txt 2>&1 &')
time.sleep(5)

# Ver toda la salida generada por localtunnel
!cat tunnel.txt
